In [8]:
## Import the package
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns 

In [9]:
#reading the dataset
df = pd.read_csv("Assignment_std.csv",index_col=None, na_values=['NA'])
df

,Date,From,To,F1,F2,F3,Output1,Output2
0,5/22/2018,6:20:12,6:23:00,-0.575665,1.999267398,-0.375573,500,3
1,5/22/2018,6:23:12,6:26:00,-0.259506,1.523251646,0.186488,500,3
2,5/22/2018,6:26:12,6:29:00,0.477280,0.89225402,0.614144,500,3
3,5/22/2018,6:29:12,6:32:00,-0.174281,1.169007365,0.406426,500,3
4,5/22/2018,6:32:12,6:35:00,0.331572,1.157937231,0.736331,500,3


In [26]:
#exploring a few first variables 
df.head()

,Date,From,To,F1,F2,F3,Output1,Output2
0,5/22/2018,6:20:12,6:23:00,-0.575665,1.999267,-0.375573,500,3
1,5/22/2018,6:23:12,6:26:00,-0.259506,1.523252,0.186488,500,3
2,5/22/2018,6:26:12,6:29:00,0.477280,0.892254,0.614144,500,3
3,5/22/2018,6:29:12,6:32:00,-0.174281,1.169007,0.406426,500,3
4,5/22/2018,6:32:12,6:35:00,0.331572,1.157937,0.736331,500,3


In [10]:
#Infomation about our data ~ Feature 2 is an Object/Categorical column 
df.columns

Index(['Date', 'From', 'To', 'F1', 'F2', 'F3', 'Output1', 'Output2'], dtype='object')

In [25]:
#EDA with pandas profiling 
pandas_profiling.ProfileReport(df)

Number of variables,8
Number of observations,709
Total Missing (%),0.0%
Total size in memory,44.4 KiB
Average record size in memory,64.1 B
Numeric,5
Categorical,3
Boolean,0
Date,0
Text (Unique),0
Rejected,0


In [7]:
# Checking if there is any missing value in the columns as a whole ~ there isnt any Missing Value 
df.isnull().any() 

Date       False
From       False
To         False
F1         False
F2         False
F3         False
Output1    False
Output2    False
dtype: bool

In [27]:
#converting F2 categorical variable to floats variables. 
cols = ['Output1', 'Output2']
for col in cols:
    df[col] = df[col].astype(dtype=np.int64)
df["F2"] = pd.to_numeric(df.F2, errors='coerce')
#df["Date"] = pd.to_numeric(df.Date, errors='coerce')

In [28]:
#Ascerting Conversion of our columns 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 709 entries, 0 to 708
Data columns (total 8 columns):
Date       709 non-null object
From       709 non-null object
To         709 non-null object
F1         709 non-null float64
F2         707 non-null float64
F3         709 non-null float64
Output1    709 non-null int64
Output2    709 non-null int64
dtypes: float64(3), int64(2), object(3)
memory usage: 44.4+ KB


In [29]:
#Cleaning dataset of NaN values 
import pandas as pd
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame), "df needs to be a pd.DataFrame"
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [30]:
#using group by to calculate average Output1 in the 4 days
grouped = df.groupby(['Date'])['Output1'].mean()
grouped

Date
5/22/2018    361.361257
5/23/2018    255.266667
5/24/2018    404.030418
5/25/2018    469.333333
Name: Output1, dtype: float64

In [17]:
#using group by to calculate average time/Output in the 4 days
gro = df.groupby(['Date'])['Output2'].mean()
gro

Date
5/22/2018    4.439791
5/23/2018    6.253333
5/24/2018    3.851711
5/25/2018    2.847619
Name: Output2, dtype: float64

In [31]:
#fitting the model
import statsmodels.formula.api as smf
X = df[['F1','F2','F3']]
y = df[['Output1']]

In [24]:
#Fitting an OLS model with F1 F2 F3 as the intercepts 
X = sm.add_constant(X)
est = smf.ols(formula = 'Output1 ~ F1 + F2 + F3', data = df).fit()
est.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                Output1   R-squared:                       0.397
Model:                            OLS   Adj. R-squared:                  0.395
Method:                 Least Squares   F-statistic:                     154.3
Date:                Thu, 18 Jul 2019   Prob (F-statistic):           7.63e-77
Time:                        08:53:34   Log-Likelihood:                -4178.2
No. Observations:                 707   AIC:                             8364.
Df Residuals:                     703   BIC:                             8383.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    370.0516      3.364    110.004      0.000     363.447     376.656
F1           -22.3958      4.814     -4.652      0.000     -31.847     -12.945
F2            36.5916      3.833      9.547      0.000      29.067      44.117
F3            60.6851      5.271     11.512      0.000      50.335      71.035
==============================================================================
Omnibus:                       25.874   Durbin-Watson:                   0.530
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               12.543
Skew:                          -0.085   Prob(JB):                      0.00189
Kurtosis:                       2.370   Cond. No.                         2.80
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
#exporting the profiling report into html report
profile = pandas_profiling.ProfileReport(df)
profile.to_file(outputfile="MFS_PROFILING REPORT.html")

In [33]:
#saving as csv
df.to_csv('mfs.csv')